---
title: Parameter calculation of available data
subtitle: Data is obtained from [FRBSTATS](https://www.herta-experiment.org/frbstats/)
author: Murthadza Aznam
date: '2022-11-23'
---

In [2]:
import pandas as pd
from pathlib import Path

## 0. Getting The Data

### 0.1 Source

This notebook uses data from [FRBSTATS](https://www.herta-experiment.org/frbstats/)

In [4]:
datapath = Path('..', 'data')
external_datapath = Path(datapath, 'raw', 'external')

In [5]:
catalog: pd.DataFrame = pd.read_csv(Path(external_datapath, 'FRBSTATS2022-11-23_population.csv'))
catalog

,frb,utc,mjd,telescope,ra,dec,l,b,frequency,dm,flux,width,fluence,snr,reference,redshift,redshift_measured,ra_error,dec_error,dm_error
0,FRB 19910730A,1991-07-30 22:24:58,48467.93400,Parkes,07:06:45.900,-43:33:00.00,254.23,-15.73,430.0,591.4,0.77,113.4,87,23,https://www.wis-tns.org/object/19910730a,0.3674,-,7,7,-
1,FRB 19920428A,1992-04-28 18:14:24,48740.76000,Parkes,17:08:60.000,-15:36:00.00,6.64,14.35,430.0,276.3,0.36,51.6,18,7.2,https://www.wis-tns.org/object/19920428a,0.1782,-,7,7,-
2,FRB 19920913A,1992-09-13 0:51:50,48878.03600,Parkes,15:03:00.000,-05:12:00.00,352.43,44.66,430.0,3337.9,0.23,157,37,8.2,https://www.wis-tns.org/object/19920913a,2.8319,-,7,7,-
3,FRB 19921212A,1992-12-12 6:10:05,48968.25700,Parkes,21:46:15.000,-07:46:60.00,47.91,-42.14,430.0,838.9,0.62,201,126,24.9,https://www.wis-tns.org/object/19921212a,0.7945,-,7,7,-
4,FRB 20010125A,2001-01-25 0:29:16,51934.02032,Parkes,19:06:53.000,-40:37:14.00,356.64,-20.02,1372.5,790.3,0.54,10.6,5.72,25,https://www.wis-tns.org/object/20010125a,0.7083,-,7.5,7.5,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
814,FRB 20220501C,2022-05-01 2:11:09,59700.09108,ASKAP,23:29:50.000,-32:27:00.00,11.26,-71.55,863.5,449.5,-,-,30,16.1,https://www.wis-tns.org/object/20220501c,0.4558,-,3,3,0.2
815,FRB 20220610A,2022-06-10 22:26:43,59740.93522,ASKAP,23:23:60.000,-33:31:00.00,8.87,-70.13,1271.5,1458.1,-,5.6,43,29.8,https://www.wis-tns.org/object/20220610a,1.3836,-,3,3,0.2
816,FRB 20220725A,2022-07-25 21:54:53,59785.91311,ASKAP,22:26:60.000,+34:52:60.00,92.14,-19.21,919.5,290.4,-,-,-,12.7,https://www.wis-tns.org/object/20220725a,0.2591,-,50,80,0.3
817,FRB 20220918A,2022-09-18 17:33:34,59840.73164,ASKAP,01:10:57.900,-70:47:05.90,300.61,-46.26,1271.5,656.8,-,-,55,26.4,https://www.wis-tns.org/object/20220918a,0.6313,-,8,8,0.4


## 1. Parameters

### 1.1 

In [16]:
rptrs = pd.read_csv(Path(external_datapath, "FRBSTATS2022-11-23_repeaters.csv"))
catalog['labels'] = ['repeater' if name in [*rptrs['name'].to_list(), *rptrs['samples'].to_list()] else 'non-repeater' for name in catalog['frb']]
catalog['labels'].value_counts()

non-repeater    612
repeater        207
Name: labels, dtype: int64